<a href="https://colab.research.google.com/github/ai-fast-track/icevision-gradio/blob/master/IceApp_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IceVision Deployment App Example: PETS Dataset
This example uses Faster RCNN trained weights using the [PETS dataset](https://airctic.github.io/icedata/pets/)

About IceVision:
- an Object-Detection Framework that connects to different libraries/frameworks such as Fastai, Pytorch Lightning, and Pytorch with more to come.
- Features a Unified Data API with out-of-the-box support for common annotation formats (COCO, VOC, etc.)
- Provides flexible model implementations with pluggable backbones

## Installing packages

In [ ]:
!pip install icevision[inference]

In [ ]:
!pip install icedata

In [ ]:
!pip install gradio

## Imports

In [4]:
from icevision.all import *
import icedata
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

## Loading trained model

In [5]:
class_map = icedata.pets.class_map()
model = icedata.pets.trained_models.faster_rcnn_resnet50_fpn()

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


Downloading: "https://github.com/airctic/model_zoo/releases/download/m3/pets_faster_resnetfpn50.zip" to /root/.cache/torch/hub/checkpoints/pets_faster_resnetfpn50.zip


## Defininig the predict() method


In [6]:
def predict(
    model, image, detection_threshold: float = 0.5, mask_threshold: float = 0.5
):
    tfms_ = tfms.A.Adapter([tfms.A.Normalize()])
    # Whenever you have images in memory (numpy arrays) you can use `Dataset.from_images`
    infer_ds = Dataset.from_images([image], tfms_)

    batch, samples = faster_rcnn.build_infer_batch(infer_ds)
    preds = faster_rcnn.predict(
        model=model,
        batch=batch,
        detection_threshold=detection_threshold
    )
    return samples[0]["img"], preds[0]

## Defining the `show_preds` method: called by `gr.Interface(fn=show_preds, ...)`

In [7]:
def show_preds(input_image, display_list, detection_threshold):
    display_label = ("Label" in display_list)
    display_bbox = ("BBox" in display_list)

    if detection_threshold==0: detection_threshold=0.5
    
    img, pred = predict(model=model, image=input_image, detection_threshold=detection_threshold)
    # print(pred)
    img = draw_pred(img=img, pred=pred, class_map=class_map, denormalize_fn=denormalize_imagenet, display_label=display_label, display_bbox=display_bbox)
    img = PIL.Image.fromarray(img)
    # print("Output Image: ", img.size, type(img))
    return img

## Gradio User Interface

In [ ]:
display_chkbox = gr.inputs.CheckboxGroup(["Label", "BBox"], label="Display")
detection_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Detection Threshold")

outputs = gr.outputs.Image(type="pil")

gr_interface = gr.Interface(fn=show_preds, inputs=["image", display_chkbox,  detection_threshold_slider], outputs=outputs, title='IceApp - PETS')
gr_interface.launch(inline=False, share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://28865.gradio.app
